In [3]:

root_dir = "output"
method_name = "standard"
benchmark = "svamp"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [5]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        api_key = open_api,
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    api_key = open_api,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [6]:
# import wandb
# wandb.login()
from agential.prompting.standard.prompting import Standard
method = Standard(
        llm=llm,
        benchmark=benchmark,
)
# run = wandb.init(
#     project=benchmark, 
#     entity="agential",
#     config={
#         "model": model,
#         "eval_model": eval_model,
#         "seed": seed,
#         "num_retries": num_retries,
#         "warming": warming,
#     },
#     group=method_name,
#     tags=[f"method={method_name}", f"model={model}", f"eval_model={eval_model}", f"seed={seed}", f"num_retries={num_retries}", f"warming={warming}"],
# )

In [1]:
from datasets import load_dataset

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("tongyx361/svamp")['test']

In [6]:
for i in data:
    assert isinstance(i['Answer'], float)

In [12]:
instance = data[4]
question = instance['Body'] + " " + instance['Question']
print(question)

28 children were riding on the bus. At the bus stop 82 children got on the bus while some got off the bus. Then there were 30 children altogether on the bus. How many more children got on the bus than those that got off?


In [15]:
from agential.eval.metrics.classification import f1, fuzzy_EM, llm_as_judge_eval, precision, recall , EM
import re

In [17]:
em_scores = []

In [18]:
index = 0 
for instance in data:
    if index > 9:
        break
    question = instance['Body'] + "\n" + instance['Question']
    answer = instance["Answer"]
    
    # Inference.
    out = method.generate(
        question=question,
        key=answer,
        num_retries=num_retries,
        warming=warming
    )
    code_str = out.answer.replace("```python", "").replace("```", "").strip()
    pred_answers, _ = safe_execute(code_string=code_str)
    try:
        pred_answer = pred_answers[0]
    except:
        pred_answer = "NaN"
    
    is_correct = int(EM(pred_answer, answer, is_numeric=True))
    # Update scores.
    em_scores.append(is_correct)

KeyboardInterrupt: 

In [19]:
em_scores

[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]